In [1]:
import lasagne
import numpy as np

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
c:\users\phil\appdata\local\programs\python\python37-32\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:


#this pulls the value of k, containing the number of channels in the output of the cnn layer, and pad_size1, pad_size2, the size of the paddings
#output_size, output_size2, and learning_rate
%run config.ipynb


### Basic Layers

In [ ]:
class DotLayer(lasagne.layers.Layer):
    def __init__(self, incoming, num_units, W=lasagne.init.Normal(0.01), **kwargs):
        super(DotLayer, self).__init__(incoming, **kwargs)
        num_inputs = self.input_shape[1]
        self.num_units = num_units
        self.W = self.add_param(W, (num_inputs, num_units), name='W')

    def get_output_for(self, input, **kwargs):
        return T.dot(input, self.W)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.num_units)

In [ ]:
class BaseLayer(lasagne.layers.Layer):
    
    def get_output_for(self, incoming):

        l_hidden_1 = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k, filter_size=(3, 3), pad = 'same',
            nonlinearity=None,
            W=lasagne.init.GlorotUniform(), name = 'base')


        l_batch_1 = lasagne.layers.BatchNormLayer(l_hidden_1)


        l_relu_1 = lasagne.layers.NonlinearityLayer(l_batch_1, nonlinearity=lasagne.nonlinearities.rectify)
        
        return l_relu_1
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], k, 6, 7)


    
def BaseLayerf(incoming):
    l_hidden_1 = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k, filter_size=(3, 3), pad = 'same',
            nonlinearity=None,
            W=lasagne.init.GlorotUniform(), name = 'base')


    l_batch_1 = lasagne.layers.BatchNormLayer(l_hidden_1)


    l_relu_1 = lasagne.layers.NonlinearityLayer(l_batch_1, nonlinearity=lasagne.nonlinearities.rectify)

    return l_relu_1     
    
    
    

In [3]:

class SkipLayer(lasagne.layers.Layer):
    def get_output_for(self, input1, input2):
        return input1 + input2

def SkipLayerf(incoming1, incoming2):
    return incoming1 + incoming2



In [4]:
class ResidualBlockLayer(lasagne.layers.Layer):
    def get_output_for(self, incoming):
        l_resid_1a = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k, filter_size=(3, 3), pad = 'same', 
            nonlinearity=None,
            W=lasagne.init.GlorotUniform(), name = 'resid')

        l_resid_1b = lasagne.layers.BatchNormLayer(l_resid_1a)

        l_resid_1c = lasagne.layers.NonlinearityLayer(l_resid_1b, nonlinearity=lasagne.nonlinearities.rectify)

        l_resid_1d = lasagne.layers.Conv2DLayer(
              l_resid_1c, num_filters=k, filter_size=(3, 3), pad = 'same', 
              nonlinearity=None,
              W=lasagne.init.GlorotUniform())

        l_resid_1e = lasagne.layers.BatchNormLayer(l_resid_1d)

        #l_resid_1f = lasgane.layers.SkipLayer(l_resid_1e, l_resid_1a)
        l_resid_1f = lasagne.layers.ElemwiseSumLayer([l_resid_1e, l_resid_1a])
        
        l_resid_1g = lasagne.layers.NonlinearityLayer(l_resid_1f, nonlinearity=lasagne.nonlinearities.rectify)

        return l_resid_1f
    
def ResidualBlockLayerf(incoming):
    l_resid_1a = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k, filter_size=(3, 3), pad = 'same', 
            nonlinearity=None,
            W=lasagne.init.GlorotUniform(), name = 'resid')

    l_resid_1b = lasagne.layers.BatchNormLayer(l_resid_1a)

    l_resid_1c = lasagne.layers.NonlinearityLayer(l_resid_1b, nonlinearity=lasagne.nonlinearities.rectify)

    l_resid_1d = lasagne.layers.Conv2DLayer(
          l_resid_1c, num_filters=k, filter_size=(3, 3), pad = 'same', 
          nonlinearity=None,
          W=lasagne.init.GlorotUniform())

    l_resid_1e = lasagne.layers.BatchNormLayer(l_resid_1d)

    #l_resid_1f = lasgane.layers.SkipLayer(l_resid_1e, l_resid_1a)
    l_resid_1f = lasagne.layers.ElemwiseSumLayer([l_resid_1e, l_resid_1a])

    l_resid_1g = lasagne.layers.NonlinearityLayer(l_resid_1f, nonlinearity=lasagne.nonlinearities.rectify)

    return l_resid_1f
    

In [5]:
class ResidualTowerLayer(lasagne.layers.Layer):
    def get_output_for(self, incoming):
        
        l_hid_1 = ResidualBlockLayerf(incoming)
        
        l_hid_2 = ResidualBlockLayerf(l_hid_1)

        l_hid_3 = ResidualBlockLayerf(l_hid_2)

        l_hid_4 = ResidualBlockLayerf(l_hid_3)

        l_hid_5 = ResidualBlockLayerf(l_hid_4)

        l_hid_6 = ResidualBlockLayerf(l_hid_5)
        
        return l_hid_6

def ResidualTowerLayerf(incoming):
    l_hid_1 = ResidualBlockLayerf(incoming)
        
    l_hid_2 = ResidualBlockLayerf(l_hid_1)

    l_hid_3 = ResidualBlockLayerf(l_hid_2)

    l_hid_4 = ResidualBlockLayerf(l_hid_3)

    l_hid_5 = ResidualBlockLayerf(l_hid_4)

    l_hid_6 = ResidualBlockLayerf(l_hid_5)

    return l_hid_6




In [20]:
class ActionHeadLayer(lasagne.layers.Layer):
    def get_output_for(self, incoming):
        l_action_head_a = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k2, filter_size=(1,1), pad = 'same', untie_biases = False,
            nonlinearity=None,
            W=lasagne.init.GlorotUniform())

        l_action_head_b = lasagne.layers.BatchNormLayer(l_action_head_a)

        l_action_head_c = lasagne.layers.NonlinearityLayer(l_action_head_b, nonlinearity=lasagne.nonlinearities.rectify)


        l_action_head_d = lasagne.layers.DenseLayer(l_action_head_c, num_units = output_size, nonlinearity=lasagne.nonlinearities.softmax)

        return l_action_head_d

    
    
def ActionHeadLayerf(incoming):
    l_action_head_a = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k2, filter_size=(1,1), pad = 'same', untie_biases = False,
            nonlinearity=None,
            W=lasagne.init.GlorotUniform())

    l_action_head_b = lasagne.layers.BatchNormLayer(l_action_head_a)

    l_action_head_c = lasagne.layers.NonlinearityLayer(l_action_head_b, nonlinearity=lasagne.nonlinearities.rectify)

    l_action_head_d = lasagne.layers.DenseLayer(l_action_head_c, num_units = output_size, nonlinearity=lasagne.nonlinearities.softmax)

    return l_action_head_d

In [21]:

class ValueHeadLayer(lasagne.layers.Layer):
    def get_output_for(self, incoming):
        l_action_head_a = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k2, filter_size=(1,1), pad = 'same', untie_biases = False,
            nonlinearity=None,
            W=lasagne.init.GlorotUniform())

        l_action_head_b = lasagne.layers.BatchNormLayer(l_action_head_a)

        l_action_head_c = lasagne.layers.NonlinearityLayer(l_action_head_b, nonlinearity=lasagne.nonlinearities.rectify)

        l_action_head_d = lasagne.layers.DenseLayer(l_action_head_c, num_units = output_size_valuehead, nonlinearity=lasagne.nonlinearities.rectify)

        l_action_head_e = lasagne.layers.DenseLayer(l_action_head_d, num_units = 1, nonlinearity=lasagne.nonlinearities.sigmoid)


        return l_action_head_e
    
def ValueHeadLayerf(incoming):
    l_action_head_a = lasagne.layers.Conv2DLayer(
            incoming, num_filters=k2, filter_size=(1,1), pad = 'same', untie_biases = False,
            nonlinearity=None,
            W=lasagne.init.GlorotUniform())

    l_action_head_b = lasagne.layers.BatchNormLayer(l_action_head_a)

    l_action_head_c = lasagne.layers.NonlinearityLayer(l_action_head_b, nonlinearity=lasagne.nonlinearities.rectify)

    l_action_head_d = lasagne.layers.DenseLayer(l_action_head_c, num_units = output_size_valuehead, nonlinearity=lasagne.nonlinearities.rectify)

    l_action_head_e = lasagne.layers.DenseLayer(l_action_head_d, num_units = 1, nonlinearity=lasagne.nonlinearities.sigmoid)


    return l_action_head_e
    
    


### Combining Layers

In [34]:
def cnn_policy1():

    l_in = lasagne.layers.InputLayer(shape=(None, 2, 6, 7))

    l_base = BaseLayerf(l_in)

    l_resid = ResidualTowerLayerf(l_base)

    l_action = ActionHeadLayerf(l_resid)

    return l_action

def cnn_policy2():

    l_in = lasagne.layers.InputLayer(shape=(None, 2, 6, 7))

    l_base = BaseLayerf(l_in)

    l_resid = ResidualTowerLayerf(l_base)

    l_action = ActionHeadLayerf(l_resid)

    return l_action

In [35]:

def cnn_value1():
    
    l_in = lasagne.layers.InputLayer(shape=(None, 2, 6, 7))

    l_base = BaseLayerf(l_in)

    l_resid = ResidualTowerLayerf(l_base)

    l_value = ValueHeadLayerf(l_resid)

    return l_value


def cnn_value2():
    
    l_in = lasagne.layers.InputLayer(shape=(None, 2, 6, 7))

    l_base = BaseLayerf(l_in)

    l_resid = ResidualTowerLayerf(l_base)

    l_value = ValueHeadLayerf(l_resid)

    return l_value

### Derivatives of the cnn functions (not in the calculus sense)

In [24]:
import theano.tensor as T
def cnn_action_1_pred(inp, network):

    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    
    
    prediction = lasagne.layers.get_output(network, input_var)
    
    #print(inp.shape)
    theano.function([input_var], prediction)(inp)
    
    
    
    
    return prediction

def cnn_action_2_pred(inp, network):

    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    
    prediction = lasagne.layers.get_output(network, input_var)
    
    theano.function([input_var], prediction)(inp)
    
    return prediction

In [25]:


def cnn_value_1_pred(inp, network):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    
    prediction = lasagne.layers.get_output(network)
    
    theano.function([input_var], prediction)(inp)
    
    return prediction


def cnn_value_2_pred(inp, network):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    
    prediction = lasagne.layers.get_output(network)
    
    theano.function([input_var], prediction)(inp)
    
    return prediction

In [26]:


def cnn_action_1_loss(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss
    

def cnn_action_2_loss(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss    



In [27]:


def cnn_value_1_loss(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss

def cnn_value_2_loss(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss



In [28]:






def cnn_action_1_grad(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')
    
    network = cnn_action1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad



def cnn_action_2_grad(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad






In [29]:


def cnn_value_1_grad(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad



def cnn_value_2_grad(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad







##### Alternative loss


In [30]:
def cnn_action_1_loss_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss

def cnn_action_2_loss_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss
    
    


In [31]:
def cnn_value_1_loss_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss


def cnn_value_2_loss_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], loss)(inp, tar)
    
    return loss


In [32]:


def cnn_action_1_grad_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad


def cnn_action_2_grad_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_action2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grad = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad


In [33]:


def cnn_value_1_grad_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value1(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grads = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad


def cnn_value_2_grad_2(inp, tar):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    network = cnn_value2(input_var)
    prediction = lasagne.layers.get_output(network)

    loss = prediction
    loss = loss.mean()

    
    params = lasagne.layers.get_all_params(network, trainable=True)
    grads = lasagne.updates.get_or_compute_grads(loss, params)
    
    
    
    
    theano.function([input_var, target_var], grad)(inp, tar)
    
    
    return grad


